In [1]:
import pandas as pd

In [2]:
hyung = pd.read_csv('./면접왕 이형 데이터 분석/면접왕이형/data.csv',engine='python',encoding='euc-kr')
hyung.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [3]:
hyung.head()

,title,run_time,date,view
0,상반기를 이렇게 준비하면 당연히 떨어집니다,7:51,2022. 2. 17.,"조회수 26,080회"
1,2월에 진짜 놓치면 안 되는 것,8:36,2022. 2. 15.,"조회수 55,310회"
2,서류에서 매번 광탈하는 취준생 특징,1:11:05,실시간 스트리밍 시작일: 2022. 2. 14.,"조회수 13,909회"
3,"공백기 2년, 스펙 부족 지원자에 대한 면접관의 속마음",9:43,2022. 2. 13.,"조회수 32,281회"
4,나도 모르게 취업이 안 되는 이유,7:17,2022. 2. 9.,"조회수 16,045회"


## 날짜 전처리

#### '실시간 스트리밍 시작일' 삭제

In [4]:
for i in range(len(hyung)):
    if '실시간 스트리밍 시작일: ' in hyung['date'][i]:
        hyung['date'][i] = hyung['date'][i][14:]

In [5]:
for i in range(len(hyung)):
    if '최초 공' in hyung['date'][i]:
        hyung['date'][i] = hyung['date'][i][7:]

#### 날짜 년, 월, 일로 나누기

In [6]:
hyung

,title,run_time,date,view
0,상반기를 이렇게 준비하면 당연히 떨어집니다,7:51,2022. 2. 17.,"조회수 26,080회"
1,2월에 진짜 놓치면 안 되는 것,8:36,2022. 2. 15.,"조회수 55,310회"
2,서류에서 매번 광탈하는 취준생 특징,1:11:05,2022. 2. 14.,"조회수 13,909회"
3,"공백기 2년, 스펙 부족 지원자에 대한 면접관의 속마음",9:43,2022. 2. 13.,"조회수 32,281회"
4,나도 모르게 취업이 안 되는 이유,7:17,2022. 2. 9.,"조회수 16,045회"
...,...,...,...,...
410,진짜 잘 본 면접: 면접 잘 본거 같아요? 그 기준이 뭘까요?,7:37,2018. 12. 5.,"조회수 223,441회"
411,"면접관이 원하는, 두괄식 답변을 알려주마! (말을 짧게 하는게 두괄식이 아니야...)",4:36,2018. 12. 5.,"조회수 221,963회"
412,"면접관이 밝히는, 면접준비의 핵심 - 수비형면접을 벗어나라",6:59,2018. 12. 5.,"조회수 97,103회"
413,면접 마인드셋: 성공하는 면접은 시작이 다르다!,2:58,2018. 12. 5.,"조회수 41,178회"


In [7]:
for i in range(len(hyung)):
    hyung['date'][i] = hyung['date'][i][:-1]
    hyung['date'][i] = hyung['date'][i].split('. ')

In [8]:
hyung

,title,run_time,date,view
0,상반기를 이렇게 준비하면 당연히 떨어집니다,7:51,"[2022, 2, 17]","조회수 26,080회"
1,2월에 진짜 놓치면 안 되는 것,8:36,"[2022, 2, 15]","조회수 55,310회"
2,서류에서 매번 광탈하는 취준생 특징,1:11:05,"[2022, 2, 14]","조회수 13,909회"
3,"공백기 2년, 스펙 부족 지원자에 대한 면접관의 속마음",9:43,"[2022, 2, 13]","조회수 32,281회"
4,나도 모르게 취업이 안 되는 이유,7:17,"[2022, 2, 9]","조회수 16,045회"
...,...,...,...,...
410,진짜 잘 본 면접: 면접 잘 본거 같아요? 그 기준이 뭘까요?,7:37,"[2018, 12, 5]","조회수 223,441회"
411,"면접관이 원하는, 두괄식 답변을 알려주마! (말을 짧게 하는게 두괄식이 아니야...)",4:36,"[2018, 12, 5]","조회수 221,963회"
412,"면접관이 밝히는, 면접준비의 핵심 - 수비형면접을 벗어나라",6:59,"[2018, 12, 5]","조회수 97,103회"
413,면접 마인드셋: 성공하는 면접은 시작이 다르다!,2:58,"[2018, 12, 5]","조회수 41,178회"


#### 년 월 일 따로 분리 (year, month, day column으로)

year = []
month = []
day = []
for i in range(len(hyung)):
    year.append(hyung['date'][i][:4])
    
    # 각 날짜에 따라 슬라이스하는 위치가 달라짐
    # 00월 00일
    if len(hyung['date'][i]) == 11:
        month.append(hyung['date'][i][6:7])
        day.append(hyung['date'][i][9:10])
    # 00월 0일 or 0월 00일
    elif len(hyung['date'][i]) == 12:
        if hyung['date'][i][7] == '.':
            month.append(hyung['date'][i][6])
            day.append(hyung['date'][i][9:11])
        elif hyung['date'][i][8] == '.':
            month.append(hyung['date'][i][6:8])
            day.append(hyung['date'][i][10:11])
    # 0월 0일
    else:
        month.append(hyung['date'][i][6:8])
        day.append(hyung['date'][i][10:12])

hyung['year'] = year
hyung['month'] = month
hyung['day'] = day

hyung.head()

## view 전처리

#### 숫자만 남기기

In [9]:
for i in range(len(hyung)):
    hyung['view'][i] = hyung['view'][i][4:-1]
    if ',' in hyung['view'][i]:
        hyung['view'][i] = hyung['view'][i].replace(',','')

hyung.head()

,title,run_time,date,view
0,상반기를 이렇게 준비하면 당연히 떨어집니다,7:51,"[2022, 2, 17]",26080
1,2월에 진짜 놓치면 안 되는 것,8:36,"[2022, 2, 15]",55310
2,서류에서 매번 광탈하는 취준생 특징,1:11:05,"[2022, 2, 14]",13909
3,"공백기 2년, 스펙 부족 지원자에 대한 면접관의 속마음",9:43,"[2022, 2, 13]",32281
4,나도 모르게 취업이 안 되는 이유,7:17,"[2022, 2, 9]",16045


#### int형으로 형변환

In [10]:
hyung['view'] = hyung['view'].astype(int)

## run_time 전처리

In [11]:
for i in range(len(hyung)):
    hyung['run_time'][i] = hyung['run_time'][i].split(':')
hyung

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,title,run_time,date,view
0,상반기를 이렇게 준비하면 당연히 떨어집니다,"[7, 51]","[2022, 2, 17]",26080
1,2월에 진짜 놓치면 안 되는 것,"[8, 36]","[2022, 2, 15]",55310
2,서류에서 매번 광탈하는 취준생 특징,"[1, 11, 05]","[2022, 2, 14]",13909
3,"공백기 2년, 스펙 부족 지원자에 대한 면접관의 속마음","[9, 43]","[2022, 2, 13]",32281
4,나도 모르게 취업이 안 되는 이유,"[7, 17]","[2022, 2, 9]",16045
...,...,...,...,...
410,진짜 잘 본 면접: 면접 잘 본거 같아요? 그 기준이 뭘까요?,"[7, 37]","[2018, 12, 5]",223441
411,"면접관이 원하는, 두괄식 답변을 알려주마! (말을 짧게 하는게 두괄식이 아니야...)","[4, 36]","[2018, 12, 5]",221963
412,"면접관이 밝히는, 면접준비의 핵심 - 수비형면접을 벗어나라","[6, 59]","[2018, 12, 5]",97103
413,면접 마인드셋: 성공하는 면접은 시작이 다르다!,"[2, 58]","[2018, 12, 5]",41178


#### 분, 초로 분해

hour = []
minute = []
second = []

for i in range(len(hyung)):
    # ':'를 기준으로 시, 분, 초 분해
    split = hyung['run_time'][i].split(':')
    
    # 영상 길이가 초
    if len(split) == 1:
        hour.append(0)
        minute.append(0)
        second.append(split[0])
        
    # 영상 길이가 분
    elif len(split) == 2:
        hour.append(0)
        minute.append(split[0])
        second.append(split[1])
        
    # 영상 길이가 시
    else:
        hour.append(split[0])
        minute.append(split[1])
        second.append(split[2])


hyung['hour'] = hour
hyung['minute'] = minute
hyung['second'] = second

#### 한자리수인 초가 두자리로 되어있음. 한자리로 바꾸기

for i in range(len(hyung)):
    if hyung['second'][i][0] == '0':
        hyung['second'][i] = hyung['second'][i][1:]
hyung

for i in range(len(hyung)):
    if hyung['second'][i][0] == '0':
        hyung['second'][i] = hyung['second'][i][1:]
hyung

## title 전처리

In [12]:
#한글 형태소 분석기인 konlpy사용 

from konlpy.tag import Kkma
from konlpy.tag import Mecab
from konlpy.tag import Twitter
#apply lambda사용
from konlpy.tag import Komoran
import re

In [13]:
kkma = Kkma()
komoran = Komoran()

In [14]:
title_ls=[]
for i in range(len(hyung)):
    special = re.compile(r'[^ A-Za-z0-9가-힣+]')
    result = special.sub('', hyung['title'].iloc[i])     
    title_ls.append(result)
    
hyung['title'] = title_ls

In [15]:
noun = []
for i in range(len(hyung)):
    noun0 = komoran.nouns(hyung['title'].iloc[i])
    noun1=[]
    for j in range(len(noun0)):
        if len(noun0[j]) ==  1:
            continue
        noun1.append(noun0[j])
    noun.append(noun1)
hyung['token'] = noun
hyung

,title,run_time,date,view,token
0,상반기를 이렇게 준비하면 당연히 떨어집니다,"[7, 51]","[2022, 2, 17]",26080,"[상반기, 준비]"
1,2월에 진짜 놓치면 안 되는 것,"[8, 36]","[2022, 2, 15]",55310,[2월]
2,서류에서 매번 광탈하는 취준생 특징,"[1, 11, 05]","[2022, 2, 14]",13909,"[서류, 특징]"
3,공백기 2년 스펙 부족 지원자에 대한 면접관의 속마음,"[9, 43]","[2022, 2, 13]",32281,"[공백기, 스펙, 부족, 지원자, 면접관, 속마음]"
4,나도 모르게 취업이 안 되는 이유,"[7, 17]","[2022, 2, 9]",16045,"[취업, 이유]"
...,...,...,...,...,...
410,진짜 잘 본 면접 면접 잘 본거 같아요 그 기준이 뭘까요,"[7, 37]","[2018, 12, 5]",223441,"[면접, 면접, 기준]"
411,면접관이 원하는 두괄식 답변을 알려주마 말을 짧게 하는게 두괄식이 아니야,"[4, 36]","[2018, 12, 5]",221963,"[면접관, 답변, 아니야]"
412,면접관이 밝히는 면접준비의 핵심 수비형면접을 벗어나라,"[6, 59]","[2018, 12, 5]",97103,"[면접관, 면접, 준비, 핵심, 수비, 면접]"
413,면접 마인드셋 성공하는 면접은 시작이 다르다,"[2, 58]","[2018, 12, 5]",41178,"[면접, 마인드, 성공, 면접, 시작]"


### 단어별 조회 수

In [16]:
# 토큰별 조회수
token_view = pd.DataFrame({'token':[]})

for i in range(len(hyung)):
    insert_data = pd.DataFrame({'token':hyung['token'].iloc[i]})
    insert_data['view'] = hyung['view'].iloc[i]
    token_view = token_view.append(insert_data)

token_view.head()

,token,view
0,상반기,26080.0
1,준비,26080.0
0,2월,55310.0
0,서류,13909.0
1,특징,13909.0


In [17]:
token_view['view'] = token_view['view'].astype('float64')
view_sum = token_view.groupby('token')['view'].sum().reset_index()

view_sum['count'] = token_view.groupby('token').count().reset_index()['view'].tolist()
view_sum

,token,view,count
0,10대,119166.0,2
1,2월,55310.0,1
2,30대,41891.0,1
3,3C,230617.0,1
4,90,21885.0,1
...,...,...,...
510,후에,36099.0,1
511,후회,191520.0,2
512,휴가,24271.0,1
513,흥미,16167.0,1


In [20]:
# 조회수 float으로 형변환
token_view['view'] = token_view['view'].astype('float64')

# 토큰으로 그룹화하고, 토큰 한 개당 조회수 합 구하기
view_sum = token_view.groupby('token')['view'].sum().reset_index()

# 해당 토큰이 나온 횟수 구하기
view_sum['count'] = token_view.groupby('token').count().reset_index()['view'].tolist()

view_count = []

for i in range(len(view_sum)):
    # 토큰의 총 조회수 / 토큰 개수 (토큰 하나당 조회수)
    a = view_sum['view'].iloc[i] / view_sum['count'].iloc[i]
    view_count.append(a)
    
view_sum['view_count'] = view_count
view_sum.sort_values(by='view',ascending = False)[:30]

,token,view,count,view_count
153,면접관,24258842.0,134,181036.134328
152,면접,13316970.0,113,117849.292035
485,합격,6802094.0,41,165904.731707
10,가지,6043876.0,34,177761.058824
227,소개,5887457.0,14,420532.642857
101,답변,4486371.0,21,213636.714286
412,질문,4442831.0,29,153201.068966
508,회사,4038248.0,10,403824.800000
394,준비,3778560.0,45,83968.000000
401,지원,3632566.0,32,113517.687500
